In [36]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name

In [78]:
model_name = None
image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/aquila-chat-7b-inference-api:latest".format(
    account_id, region_name
)
base_name = sagemaker.utils.base_name_from_image(image_uri)

In [79]:
model_data = f"s3://{bucket}/model/aquila-chat-7b/model.tar.gz"
print(model_data)
# !touch dummy
# !tar czvf model.tar.gz dummy
# !rm dummy
# !aws s3 cp model.tar.gz $model_data

s3://sagemaker-us-east-2-946277762357/model/aquila-chat-7b/model.tar.gz


In [90]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Model(
    name=model_name,
    model_data=model_data,
    role=role,
    image_uri=image_uri,
    predictor_cls=Predictor,
)

In [91]:
instance_type = "ml.g5.4xlarge"
instance_count = 1

In [ ]:
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=instance_count,
    container_startup_health_check_timeout=1200,
)

In [85]:
print(predictor.endpoint_name)

aquila-chat-7b-inference-api-2023-06-19-07-38-17-031


In [86]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs = {
    "inputs": "MSK有什么优势？",
    "max_length":200,
    "history": []
}

prediction = predictor.predict(inputs)

In [87]:
prediction['response']

'作为AI语言模型，我可以告诉你一些MSK的优势：\n\n1. 自然语言处理：MSK可以处理大量的文本数据，并生成自然流畅的回复，这使得它在各种场景下都能提供高效、准确的服务。\n\n2. 多语言支持：MSK支持多种语言，包括中文、英语、法语、德语、俄语等，这使得它可以更好地满足全球客户的需求。\n\n3. 快速响应：MSK可以快速处理大量的文本数据，并在短时间内生成回复，这使得它在一些紧急情况下能够提供快速、高效的解决方案。\n\n4. 智能对话：MSK可以学习和理解大量的对话数据，并生成智能对话回复，这使得它可以更好地满足客户的需求，并提供更个性化的服务。\n\n总之，MSK具有高效、准确、快速、智能的特点，这些优势使得它在各种场景下都能提供优质的服务。\n'

In [89]:
#删除模型节点
# predictor.delete_endpoint()